Import packages and read in the college confidential data.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

pd.set_option('display.max_colwidth', None)
pd.options.mode.chained_assignment = None

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

df = pd.read_csv("gdrive/My Drive/Dissertation Complete/college_confidential_cleaned.csv")

df['university'] = np.nan
df.head()

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


,content,university
0,Bump?,NaN
1,"About this semester, don't let the semester end without seeing if you can get a medical withdrawal ""W"" for one or more of your current courses that you are failing. Do you really need the prof's ok for a W? Your advisor should be helping with this. But if you are receiving financial aid, you might have to keep a minimum number of credits. Good luck and congratulations! I",NaN
2,"Since none of the professors you and your adviser reached out to responded, ask your adviser to seek a medical withdrawal from the classes you're getting D's or F's in, and as CheddarcheeseMN mentioned, you need to keep 12 credits BUT if you get medical approval you can go below that, not clear on administrative rules but I think you can go to 8 without penalty for a semester for medical reasons ONLY (not sure pregnancy counts but you can look into it with your adviser). However, ACT IMMEDIATELY, as once the semester's over, you can't go back on your grade etc. So it needs to be fixed NOW. Your plan is good theoretically but I'm not sure how you'll be able to handle college with an infant: do you have care solutions (ie., a grandparent who can stay at home with the baby?)",NaN
3,"I would imagine pregnancy would be treated differently because it's a choice. I hope you reconsider following through with this, because having a child so early can be a life-ruiner. Temple's transfer acceptance rate was like 81% last I checked, but I'd recommend trying to re-take the courses you failed so you can replace the Fs with better grades. Temple's in-state tuition is like 20,000 a year. And any debt you take on for a degree in a low-pay field like social work will haunt you because you'll have infant expenses.",NaN
4,"Well,it's not an illness, it's also a medical condition (your body gets distorted inside, organs are crushed, your hormones are totally whack, you can't stand on your legs, you need rest you wouldn't normally need, medicine and vitamins.) That's why pregnancy is covered by health insurance: it affects your health.",NaN


In [ ]:
df.shape

(3769015, 2)

Parse the dataset in sections due to google collab runtime disconnects

In [ ]:
df = df[3500000:]
df.reset_index(inplace=True, drop=True)

df.head()

,content,university
0,Syllabli are used to determine how many credits you will receive/,NaN
1,I submitted a Midterm report.,NaN
2,I also applied to Questrom! What are your majors?,NaN
3,I applied to COM. Journalism major here!,NaN
4,"If you don't mind me asking, which midterm report form did you send in? The CommonApp one or the BU one ( )? And did you just send them an email? Or is there an online form to submit it?",NaN


Read in the actual ranking list, to extract the names of the top 50 universities

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Actual_ranking = pd.read_csv("gdrive/My Drive/Dissertation Complete/National Universities Rankings.csv", encoding='latin-1')
Actual_ranking = Actual_ranking[["Name", "Rank"]]

Actual_ranking.head()

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


,Name,Rank
0,Princeton University,1
1,Harvard University,2
2,University of Chicago,3
3,Yale University,3
4,Columbia University,5


In [ ]:
top_50_unis = Actual_ranking['Name'][0:50]
top_50_unis[0]

'Princeton University'

Named Entity Recognition

In [ ]:
import spacy
import en_core_web_sm

nlp = spacy.load('en_core_web_sm')

Here we manually create abbreviations for the top 50 universities so that the NER system can easily detect them rather than large strings.

In [ ]:
top_X_Unis = [
    "Princeton",
    "Harvard",
    "UChicago",
    "Yale",
    "Columbia",
    "Stanford",
    "MIT",
    "Duke",
    "Penn",
    "Johns Hopkins",
    "Dartmouth",
    "Caltech",
    "Northwestern",
    "Brown",
    "Cornell",
    "Rice",
    "Notre Dame",
    "Vanderbilt",
    "Washington",
    "Emory",
    "Georgetown",
    "Berkeley",
    "USC",
    "Carnegie Mellon",
    "UCLA",
    "UVA",
    "Tufts",
    "UMich",
    "Wake Forest",
    "UNC",
    "Boston College",
    "College of William & Mary",
    "Rochester",
    "Brandeis",
    "Georgia Institute of Technology",
    "New York University",
    "Case Western Reserve University",
    "University of California--Santa Barbara",
    "Boston University",
    "Northeastern University",
    "Rensselaer Polytechnic Institute",
    "Tulane",
    "University of California--Irvine",
    "Lehigh",
    "University of California--Davis",
    "University of California--San Diego",
    "University of Illinois--Urbana-Champaign",
    "University of Miami",
    "University of Wisconsin--Madison",
    "Pepperdine"
]

# Counter for the dataframe, as the loop iterates by row, we update the position so that the NE can be inserted to the correct row
i = 0

# for each row of the dataframe
for text in df["content"]:
    doc = nlp(text) # Create the Doc object by passing it through the text pipeline (nlp)

    # Run NER on the Doc object
    for ne in doc.ents:
      t = text[ne.start_char:ne.end_char] # text position in string of NE

      # If the entity label is of type organisation
      #if ne.label_ == 'ORG':

      # If the text of the named entity matches the universities in the list, insert the named entity to the dataframe
      if t in top_X_Unis:
        df['university'][i] = t
    i+=1

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 269015 entries, 0 to 269014
Data columns (total 2 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   content     269015 non-null  object
 1   university  20164 non-null   object
dtypes: object(2)
memory usage: 4.1+ MB


In [ ]:
df.shape

(269015, 2)

drop all rows that do not have a university associated with it

In [ ]:
df.dropna(inplace=True)
df.shape

(20164, 2)

List all of the universities found

In [ ]:
df['university'].value_counts()

Columbia                            1629
Stanford                            1327
Boston College                      1179
Harvard                             1160
USC                                  951
UCLA                                 941
Cornell                              931
MIT                                  871
Georgetown                           861
Yale                                 760
Tufts                                758
Dartmouth                            583
Berkeley                             577
Penn                                 563
Brown                                523
Duke                                 466
Tulane                               455
UVA                                  449
Boston University                    423
Princeton                            406
Washington                           403
Rice                                 401
Emory                                387
Northwestern                         353
Notre Dame      

Write the data to a NE file

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')
path = '/content/gdrive/My Drive/Dissertation Complete/college_confidential_NE.csv'

with open(path, 'a') as f:
  df.to_csv(f, encoding='utf-8', index=False, header=False)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


This is just to confirm the parsed sections have ran correctly (google collab) please ignore

In [ ]:
df = pd.read_csv("gdrive/My Drive/Dissertation Complete/college_confidential_NE.csv")

In [ ]:
df.tail()

,content,university
444617,and why did Cornell have to put up nets under its bridges ?,Cornell
444618,Because Cornell has lots of bridges.,Cornell
444619,"When a lifelong friend of my brother was an undergrad at Cornell, he invited my brother to be a guest at his fraternity house and attend a themed fraternity party. My brother's friend left the party unnoticed and committed suicide by jumping off a bridge. No one knows why. My brother were devastated. The next year, I lost a friend at MIT I'll assume that your question was not intended to sound flippant. No university is exempt from the unexplainable and heartbreaking suicides of promising young adults.",MIT
444620,Heard from a Yalie who said Princeton's parties are pretty lame. His actual experience.,Princeton
444621,"Absolutely most top schools have their fair share of suicides. It is a difficult adjustment process. Many people struggle to make peace with the fact that while they were top of their class in high school but are now just average at a place like Penn or MIT or Cornell. Also the workload and the pressure to have it together and achieve at such a high level can be a lot to handle on top of all the issues every young adult faces. At least for me what helped at Penn was the ability to have a great social circle through the many extracurricular activities, clubs, organizations offered on campus and the ability to escape from campus into Philly from time to time. There are people who prefer a smaller, less high-octane environment where they can be a bit more coddled and eased into the whole college experience. There is no shame in being one of these people but places like Penn, Cornell, MIT, Columbia are probably not the right choices for these people. I just think you need to have a sense of who you are and what environment fits you best.",Columbia


In [ ]:
df.shape

(444622, 2)

In [ ]:
df['university'].value_counts()

Harvard                                31250
MIT                                    27126
Stanford                               26310
USC                                    25423
UCLA                                   22539
Yale                                   22152
Cornell                                19902
Columbia                               18207
Princeton                              17831
Brown                                  17822
Duke                                   17718
Rice                                   15917
Tulane                                 15842
Penn                                   15810
Dartmouth                              15266
Emory                                  13957
Berkeley                               12467
Tufts                                  12275
UVA                                    11457
UNC                                    11320
Georgetown                             10751
Caltech                                 8921
Vanderbilt